In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from LeNet import LeNet

In [2]:
transform=transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor()
])
train_set=torchvision.datasets.CIFAR10(root="../data",train=True,transform=transform,download=False)
test_set=torchvision.datasets.CIFAR10(root="../data",train=False,transform=transform,download=False)

In [3]:
# img,label=train_set[0]
# img.shape,label

In [4]:
batch_size=256
lr=0.1
epoch=20
writer=SummaryWriter("logs")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
trainLoader=DataLoader(train_set,batch_size=batch_size,shuffle=True,num_workers=0,drop_last=False)
testLoader=DataLoader(test_set,batch_size=batch_size,shuffle=True,num_workers=0,drop_last=False)

In [5]:
model=LeNet().to(device)
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        nn.init.xavier_uniform_(m.weight)
model.apply(init_weights)
loss=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=lr,momentum=0.9)

In [6]:
def train(trainLoader,model,loss_fn,optimizer):
    print(f"training on:{device}")
    # model=model.to(device)
    size=len(trainLoader.dataset)
    
    for batch,(X,y) in enumerate(trainLoader):
        X=X.to(device)
        y=y.to(device)
        pred=model(X)
        loss=loss_fn(pred,y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch%10==0:
            loss,current=loss.item(),batch*len(X)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")

In [7]:
max_correct=0
def test(testLoader, model, loss_fn,max_correct):
    # model = model.to(device)
    size = len(testLoader.dataset)
    num_batches = len(testLoader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in testLoader:
            X = X.to(device)
            y = y.to(device)

            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches

    correct /= size
    
    if correct>max_correct:
        torch.save(model.state_dict(),'mlp.params')
        max_correct=correct

    print(f"test error:\n accuracy:{(100 * correct):>0.1f}%,avg loss: {test_loss:>8f} \n")
    return max_correct,correct

In [8]:
for t in range(epoch):
    print(f"epoch {t+1} \n----------------------------------------------")
    train(trainLoader,model,loss,optimizer)
    max_correct,correct=test(testLoader,model,loss,max_correct)
    writer.add_scalar("accuracy",correct,t)
    
writer.close()

epoch 1 
----------------------------------------------
training on:cuda
loss:2.593022 [    0/50000]
loss:2.306213 [  640/50000]
loss:2.324021 [ 1280/50000]
loss:2.323118 [ 1920/50000]
loss:2.322331 [ 2560/50000]
loss:2.312983 [ 3200/50000]
loss:2.302548 [ 3840/50000]
loss:2.292437 [ 4480/50000]
loss:2.292272 [ 5120/50000]
loss:2.290393 [ 5760/50000]
loss:2.299395 [ 6400/50000]
loss:2.296983 [ 7040/50000]
loss:2.297990 [ 7680/50000]
loss:2.328382 [ 8320/50000]
loss:2.291830 [ 8960/50000]
loss:2.293055 [ 9600/50000]
loss:2.304286 [10240/50000]
loss:2.309412 [10880/50000]
loss:2.277182 [11520/50000]
loss:2.329027 [12160/50000]
loss:2.299432 [12800/50000]
loss:2.285431 [13440/50000]
loss:2.286742 [14080/50000]
loss:2.329300 [14720/50000]
loss:2.288618 [15360/50000]
loss:2.311886 [16000/50000]
loss:2.324358 [16640/50000]
loss:2.316652 [17280/50000]
loss:2.288066 [17920/50000]
loss:2.299737 [18560/50000]
loss:2.302201 [19200/50000]
loss:2.301362 [19840/50000]
loss:2.309028 [20480/50000]
los